In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)


!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -U geometric-smote

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import copy
import torch
import warnings
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt



from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import resample 

from torch_geometric.utils import to_networkx
from torch_geometric.data import Data, DataLoader

import torch.nn.functional as F
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
import torch.nn as nn
from torch_geometric.nn import *


warnings.filterwarnings('ignore')

In [4]:
class Config:
    seed = 0
    learning_rate = 0.001
    weight_decay = 1e-5
    input_dim = 165
    output_dim = 1
    hidden_size = 128
    num_layers = 3
    num_epochs = 100
    checkpoints_dir = './models/elliptic_gnn'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
print("Using device:", Config.device)

Using device: cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# DATA LOADING/PREPARATION
df_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/elliptic_bitcoin_dataset/elliptic_txs_features.csv', header=None)
df_edges = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
df_classes =  pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
df_classes['class'] = df_classes['class'].map({'unknown': 2, '1': 1, '2': 0})

In [ ]:
print(df_features.shape)
df_features.head()
#df_features[df_features[0].isna()].head()

In [ ]:
df_edges.head()
#df_edges[df_edges['txId2'].isna()].head()

In [ ]:
# Hay dos problemas: 
# - Está desbalanceado y hay que oversamplear la clase 1 para que iguale las muestras de la clase 0.
# - La clase 2 es desconocida por lo que solo aporta ruido. Hay que eliminar todo rastro de esa clase antes de oversampling
print(df_classes['class'].value_counts())
#df_classes[df_classes['txId'].isna()].head()

In [ ]:
# PASO 1: Eliminar todo rastro de la clase desconocida '2' para evitar ruido y poder oversamplear
# 1.1 Quitamos los nodos con clase 2 del df_classes (fácil)
# Quitamos los nodos de la clase desconocida
df_classes2 = df_classes[df_classes['class'] != 2 ] 
print(df_classes2.shape)
print(df_classes2['class'].value_counts())



In [ ]:
# 1.2 Quitamos los nodos con clase 2 del df_features (fácil). Hacemos join con df_classes para saber la clase y suprimirla
print("df_features antes: ", df_features.shape)
df_features2 = pd.merge(df_features,df_classes2,how='inner', left_on=[0], right_on=['txId']).drop(['txId'], axis=1)
print("df_features despues: ", df_features2.shape)
df_features2.head()

In [ ]:
# 1.3 Quitamos todas las aristas que tengan algo que ver con la clase 2 (menos fácil). Lo hacemos mediante 2 outer joins (uno contra los nodos origen y otro contra los nodos destino) de la clase desconocida 
# Quitamos los edges donde están involucrados nodos de la clase desconocida
print("df_edges antes: ", df_edges.shape)
df_classes_unknown = df_classes.loc[df_classes['class'] == 2, 'txId']
print("Número de nodos pertenecientes a la clase desconocida: ", df_classes_unknown.shape)
df_edges2 = pd.merge(df_classes_unknown, df_edges, indicator=True, how = 'outer', left_on=['txId'], right_on=['txId1']).query('_merge=="right_only"').drop(['txId', '_merge'], axis=1)
print("df_edges sin clase 2 en los nodos origen: ", df_edges2.shape)
df_edges2 = pd.merge(df_classes_unknown, df_edges2, indicator=True, how = 'outer', left_on=['txId'], right_on=['txId2']).query('_merge=="right_only"').drop(['txId', '_merge'], axis=1)
print("df_edges sin clase 2 en los nodos origen ni destino: ", df_edges2.shape)
df_edges2.head()
df_edges2 = df_edges2.astype(int)
# Vemos que una vez se ha eliminado todo rastro de la clase 2, nos encontramos con que hay menos aristas que nodos de las clase 1 y 2 juntas: 36624 vs 46564

In [ ]:
# PASO 2: Una vez se ha "limpiado" la clase 2, es necesario fusionar los dfs para tener una única tabla que será la entrada del SMUTE (oversampler)
# 2.1 Añadimos al df_features la columna de clase (fácil). Debería resultar en un mismo número de filas (df_features2)

# 2.2 Añadimos la columna 'destino' al df_features (alias txId2). 
# El resultado será un aumento en el número de filas ya que un nodo puede ser origen de varias aristas.
# Convertimos los NaN a 0 para no tener problemas. Los NaN solo aparecerán en la columna 'destino' porque hay nodos que no son origen de ninguna arista
df_join = pd.merge(df_features2, df_edges2, how = 'left', left_on=[0], right_on=['txId1']).drop('txId1', axis=1).drop_duplicates()
df_join['txId2'].fillna(0, inplace=True)
print("El df resultado tiene la siguiente forma: ", df_join.shape)
print("La distribución de clase en formato tabular seria:\n",df_join['class'].value_counts())
df_join.head()

In [ ]:
# PASO 3. Una vez tenemos los datos en forma tabular, los dividimos para la entrada en el oversampler:
X = df_join.rename(columns={'txId2':len(df_join.columns)-2}).drop('class', axis=1)
y = df_join['class']
print("La entrada X tendría la siguiente forma: ", X.shape)
print("La entrada y tendría la siguiente forma: ", y.shape)
print("Distribución de y:\n", y.value_counts())
X.head()

In [ ]:
# PASO 4. Realizamos el OVERSAMPLING.
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import pandas as pd
from sklearn.impute import SimpleImputer
# Aumentamos la clase menor con G-SMOTE
from gsmote import GeometricSMOTE

ss_over = 0.99
ss_under = 0.5

# Definir la estrategia de sobremuestreo y submuestreo
over = SMOTE(sampling_strategy=ss_over)
#over = GeometricSMOTE(k_neighbors=2, selection_strategy='combined', random_state=5)
under = RandomUnderSampler(sampling_strategy=ss_under)

# Combinar ambas estrategias en un pipeline , ('o', over), ('u', under)
steps = [ ('u', under), ('o', over)]  
pipeline = Pipeline(steps=steps)

X_res, y_res = pipeline.fit_resample(X, y)

print("Resultado oversampling tabular:\n", y_res.value_counts())


In [ ]:
# PASO 5. VOLVER AL FORMATO GRAFO
# 5.1 JUNTAMOS dfs para incluir el campo class
print(X_res.shape)
X_res['class'] = y_res
X_res[167] = X_res[167].astype(int)
X_res = X_res.drop_duplicates(subset=[0, 'class'], keep=False)
print(X_res.shape)
X_res.head()

In [ ]:
# 5.2 Reconstruimos df de classes
# 1. Montamos df de classes cogiendo tanto la columna "origen" (txId1 alias 0) como la columna "destino" (txId2 alias 167). Eliminamos todas las apariciones de los nodos con más
df_c_1 = X_res[[0]].drop_duplicates().rename(columns={0:'txId'}).sort_values('txId').reset_index(drop=True)
df_c_2 = X_res[[167]].drop_duplicates().rename(columns={167:'txId'}).sort_values('txId').reset_index(drop=True)
df_nu = pd.concat([df_c_1, df_c_2]).drop_duplicates(subset=['txId'], keep='first')
#print("Nodos unicos: ", df_nu.shape)

# 2. Obtenemos las clases de los nodos unicos
df_classes_smute = pd.merge(df_nu, X_res[[0, 'class']], how = 'inner', left_on=['txId'], right_on=[0]).drop([0], axis=1).drop_duplicates(subset=['txId'], keep='first').dropna(how='any').reset_index(drop=True)
print("df_classes_smute: ", df_classes_smute.shape)
print("Distribución df_classes_smute:\n", df_classes_smute['class'].value_counts())

# 3. Obtener el complementario df_classes_smute para mas adelante. Todos los nodos creados pero que o bien tienen varias clases o bien no tienen porque son solo destinos
# pd.merge(df_classes_unknown, df_edges, indicator=True, how = 'outer', left_on=['txId'], right_on=['txId1']).query('_merge=="right_only"').drop(['txId', '_merge'], axis=1)
df_classes_smute_c = pd.merge(df_classes_smute['txId'], df_nu, indicator=True, how='outer', left_on=['txId'], right_on=['txId']).query('_merge=="right_only"'). drop(['_merge'], axis=1).reset_index(drop=True).drop_duplicates(subset=['txId'], keep='first')
print("Complementario df_classes_smute (solo nodos): ", df_classes_smute_c.shape)
# quitamos el nodo destino 0 que es "la nada"


df_classes_smute_c.head()



In [ ]:
# 5.3 Reconstruimos df de features
df_features_smute = pd.merge(df_classes_smute['txId'], X_res.drop([167, 'class'], axis=1), how = 'inner', left_on=['txId'], right_on=[0]).drop(['txId'], axis=1).drop_duplicates(subset=[0], keep='first').reset_index(drop=True)
print("df_features_smute: ", df_features_smute.shape)
#df_features_smute.head()

# Eliminamos del df_classes_smute aquellos nodos que no tengan features:
#df_classes_smute = pd.merge(df_classes_smute, df_features_smute[0], how='inner', left_on=['txId'], right_on=[0]).drop([0], axis=1).reset_index(drop=True)
print("df_classes_smute: ", df_classes_smute.shape)
#df_features_smute.head()


In [ ]:
# 5.4 Reconstruimos df de edges

# VOY POR AQUIII

df_edges_smute1 = pd.merge(df_classes_smute_c, X_res[[0, 167]], indicator=True, how = 'outer', left_on=['txId'], right_on=[0]).query('_merge=="right_only"').drop(['txId', '_merge'], axis=1)
print("df_edges_smute eliminando procesando origenes : ", df_edges_smute1.shape)
df_edges_smute2 = pd.merge(df_classes_smute_c, df_edges_smute1, indicator=True, how = 'outer', left_on=['txId'], right_on=[167]).query('_merge=="right_only"').drop(['txId', '_merge'], axis=1)
print("df_edges_smute procesando origen y destino", df_edges_smute2.shape)
df_edges_smute2 = df_edges_smute2.rename(columns={0:'txId1', 167: 'txId2'}).astype(int).reset_index(drop=True)
df_edges_smute2.head()


# Montamos el df de edges
# , cogiendo solo los nodos de los datos sinteticos que tenian una unica clase
#df_e_1 = pd.merge(df_c['txId'], X_res, how = 'left', left_on=['txId'], right_on=[0]).drop([0], axis=1)

# renombramos e imprimimos
#df_e = df_e_1[['txId', 167]].rename(columns={'txId':'txId1', 167:'txId2'})
#df_e = df_e.sort_values('txId1').reset_index(drop=True)
#print(df_e.shape)
#df_e.head()


In [ ]:
# ELIMINAR ESTO:Montamos df de features
# , cogiendo solo los nodos de los datos sinteticos que tenian una unica clase
#df_f_1 = pd.merge(df_c['txId'], X_res, how = 'inner', left_on=['txId'], right_on=[0]).drop([0], axis=1)
#df_f_2 = pd.merge(df_c['txId'], X_res, how = 'left', left_on=['txId'], right_on=[167]).drop([0], axis=1)
#df_f_3 = pd.concat([df_f_1, df_f_2]).drop_duplicates().dropna(how='any')

#df_f = df_f_3.drop([167, 'class'], axis=1).rename(columns={'txId':0}).drop_duplicates().sort_values(0).reset_index(drop=True)

#print(df_f.shape)
#df_f.head()

#HASTA UNA DUDA: SALEN MÁS ELEMENOS EN df_f que en df_c y no puede ser. No entiendo porqué. Hacer un outer join para ver porqué .query('_merge=="right_only"')
#df_prueba = pd.merge(df_c, df_f, indicator=True, how = 'outer', left_on=['txId'], right_on=[0]).drop_duplicates()
#print(df_prueba['_merge'].value_counts())
#df_prueba.head()

In [ ]:
# merging node features DF with classes DF
df_merge = df_features_smute.merge(df_classes_smute, how='inner', right_on="txId", left_on=0)
df_merge = df_merge.sort_values(0).reset_index(drop=True)

# extracting classified/non-classified nodes
classified = df_merge.loc[df_merge['class'].loc[df_merge['class']!=2].index].drop('txId', axis=1)
unclassified = df_merge.loc[df_merge['class'].loc[df_merge['class']==2].index].drop('txId', axis=1)

# extracting classified/non-classified edges
classified_edges = df_edges_smute2.loc[df_edges_smute2['txId1'].isin(classified[0]) & df_edges_smute2['txId2'].isin(classified[0])]
unclassifed_edges = df_edges_smute2.loc[df_edges_smute2['txId1'].isin(unclassified[0]) | df_edges_smute2['txId2'].isin(unclassified[0])]

In [ ]:
df_merge = df_merge.dropna()
df_merge = df_merge.reset_index(drop = True)
print(df_merge.shape)
df_merge.head()

In [ ]:
#PREPARING EDGES
# mapping nodes to indices
nodes = df_features_smute[0].values
map_id = {j:i for i,j in enumerate(nodes)}


# mapping edges to indices
edges = df_edges_smute2.copy()
edges.txId1 = edges.txId1.map(map_id)
edges.txId2 = edges.txId2.map(map_id)
edges = edges.dropna()
edges = edges.astype(int)


edge_index = np.array(edges.values).T
edge_index = torch.tensor(edge_index, dtype=torch.long).contiguous()


# weights for the edges are equal in case of model without attention
weights = torch.tensor([1] * edge_index.shape[1] , dtype=torch.float32)

print("Total amount of edges in DAG:", edge_index.shape)


In [ ]:
#PREPARING NODES
# maping node ids to corresponding indexes
node_features = df_merge.drop(['txId'], axis=1).copy()
node_features[0] = node_features[0].map(map_id)


classified_idx = node_features['class'].loc[node_features['class']!=2].index
unclassified_idx = node_features['class'].loc[node_features['class']==2].index

# replace unkown class with 0, to avoid having 3 classes, this data/labels never used in training
node_features['class'] = node_features['class'].replace(2, 0) 

labels = node_features['class'].values

# drop indeces, class and temporal axes 
node_features = torch.tensor(np.array(node_features.drop([0, 'class', 1], axis=1).values, dtype=np.float32), dtype=torch.float32)


In [ ]:
#PyG DATASET
# converting data to PyGeometric graph data format
elliptic_dataset = Data(x = node_features, 
                        edge_index = edge_index, 
                        edge_attr = weights,
                        y = torch.tensor(labels, dtype=torch.float32)) 

print(f'Number of nodes: {elliptic_dataset.num_nodes}')
print(f'Number of node features: {elliptic_dataset.num_features}')
print(f'Number of edges: {elliptic_dataset.num_edges}')
print(f'Number of edge features: {elliptic_dataset.num_features}')
print(f'Average node degree: {elliptic_dataset.num_edges / elliptic_dataset.num_nodes:.2f}')
print(f'Number of classes: {len(np.unique(elliptic_dataset.y))}')
print(f'Has isolated nodes: {elliptic_dataset.has_isolated_nodes()}')
print(f'Has self loops: {elliptic_dataset.has_self_loops()}')
print(f'Is directed: {elliptic_dataset.is_directed()}')

In [ ]:
y_train = labels[classified_idx]

# spliting train set and validation set
_, _, _, _, train_idx, valid_idx = \
    train_test_split(node_features[classified_idx], 
                     y_train, 
                     classified_idx, 
                     test_size=0.15, 
                     random_state=Config.seed, 
                     stratify=y_train)
                     
elliptic_dataset.train_idx = torch.tensor(train_idx, dtype=torch.long)
elliptic_dataset.val_idx = torch.tensor(valid_idx, dtype=torch.long)
elliptic_dataset.test_idx = torch.tensor(unclassified_idx, dtype=torch.long)

print("Train dataset size:", elliptic_dataset.train_idx.shape[0])
print("Validation dataset size:", elliptic_dataset.val_idx.shape[0])
print("Test dataset size:", elliptic_dataset.test_idx.shape[0])

In [ ]:
#TRAIN/TEST HELPERS
def train_evaluate(model, data, criterion, optimizer, *args):
    num_epochs = args[0]
    checkpoints_dir = args[1]
    model_filename = args[2]

    best_model_wts = copy.deepcopy(model.state_dict())

    best_loss = 10e10

    if not os.path.exists(checkpoints_dir):
        os.makedirs(checkpoints_dir)

    model.train()
    for epoch in range(num_epochs+1):
        # Training
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_idx], data.y[data.train_idx].unsqueeze(1))
        acc = accuracy(out[data.train_idx], data.y[data.train_idx].unsqueeze(1), prediction_threshold=0.5)
        loss.backward()
        optimizer.step()

        # Validation
        val_loss = criterion(out[data.val_idx], data.y[data.val_idx].unsqueeze(1))
        val_acc = accuracy(out[data.val_idx], data.y[data.val_idx].unsqueeze(1), prediction_threshold=0.5)

        if(epoch % 10 == 0):
            #print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
            #      f'{acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
            #      f'Val Acc: {val_acc*100:.2f}%')
        
            if val_loss < best_loss:
                best_loss = val_loss
                #print("Saving model for best loss")
                checkpoint = {
                    'state_dict': best_model_wts
                }
                torch.save(checkpoint, os.path.join(checkpoints_dir, model_filename))
                best_model_wts = copy.deepcopy(model.state_dict())

    return model

def test(model, data):
    model.eval()
    out = model(data.x, data.edge_index) 
    preds = ((torch.sigmoid(out) > 0.5).float()*1).squeeze(1)
    return preds

In [ ]:
def accuracy(y_pred, y_test, prediction_threshold=0.5):
    y_pred_label = (torch.sigmoid(y_pred) > prediction_threshold).float()*1

    correct_results_sum = (y_pred_label == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]

    return acc

In [ ]:
# Pruebas: General Model Class Implementation
import torch_geometric.nn.conv as conv
import inspect
class GNNGeneralModel(torch.nn.Module):
    
    def __init__(self, dim_in, dim_h, dim_out, model, num_layers=2):
        super(GNNGeneralModel, self).__init__()

        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        self.layers.append(conv.ClusterGCNConv(in_channels=dim_in, out_channels=dim_h))

        if self.num_layers > 2:
          for i in range(1,self.num_layers-1,1):
              layer = GENConv(in_channels=dim_h, out_channels=dim_h) 
              self.layers.append(layer)

        layer = GATv2Conv(in_channels=dim_h, out_channels=dim_out)
        self.layers.append(layer)

    def forward(self, x, edge_index):

        for i in range(0,self.num_layers-1,1):
          h = self.layers[0](x, edge_index)
          h = torch.relu(h)
          h = F.dropout(h, p=0.6, training=self.training)


        out = self.layers[len(self.layers)-1](h, edge_index)
        return out

    def get_num_layers(self):
      return len(self.layers)

In [ ]:
# Bucle funcionando

import torch_geometric.nn.conv as conv
import inspect
#import pylibcugraphops

# Obtenemos todas las clases del módulo conv
conv_classes = inspect.getmembers(conv, inspect.isclass)


# Seleccionamos sólo las clases que heredan de torch.nn.Module y tienen un método forward
conv_layers = [layer[1] for layer in conv_classes if issubclass(layer[1], torch.nn.Module) and 
               hasattr(layer[1], 'forward') and 
               'in_channels' in inspect.signature(layer[1].__init__).parameters and
               'out_channels' in inspect.signature(layer[1].__init__).parameters]

conv_layers = [layer[1] for layer in conv_classes if issubclass(layer[1], torch.nn.Module) and 
               hasattr(layer[1], 'forward') ]

layers_not_used = []
reports = {}
fraud_pcnt = {} 
# Creamos un modelo con cada capa convolucional en la lista
for conv_layer in conv_layers:

    #params = inspect.signature(conv_layer.__init__).parameters
    #sig = inspect.signature(conv_layer.__init__)
    #required_params =  {name: param for name, param in sig.parameters.items() if param.default == inspect.Parameter.empty and name != "self" and name != "kwargs"}
    #required_params_set= [name for name in required_params.keys()]

    try:
      #model = torch.nn.Sequential(
      #    conv_layer(in_channels=16, out_channels=32),
      #    conv_layer(in_channels=32, out_channels=64),
      #    conv_layer(in_channels=64, out_channels=128)
      #).to(Config.device)
      
      model = GNNGeneralModel(Config.input_dim, Config.hidden_size, Config.output_dim, conv_layer, Config.num_layers).to(Config.device)
      #print(model.get_num_layers())
      #print(model)

      data_train = elliptic_dataset.to(Config.device)

      optimizer = torch.optim.Adam(model.parameters(), lr=Config.learning_rate, weight_decay=Config.weight_decay)
      scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
      criterion = torch.nn.BCEWithLogitsLoss()
      train_evaluate(model,
              data_train,
              criterion,
              optimizer,
              Config.num_epochs,
              Config.checkpoints_dir,
              type(conv_layer).__name__ + '.pth.tar')
      
      print(model)
      model.load_state_dict(torch.load(os.path.join(Config.checkpoints_dir, type(conv_layer).__name__ + '.pth.tar'))['state_dict'])

      y_test_preds = test(model, data_train)

      # confusion matrix on validation data
      #aux_conf_mat = data_train.y[data_train.val_idx].detach().cpu()
      #conf_mat = confusion_matrix(aux_conf_mat.numpy(), y_test_preds[valid_idx])
      conf_mat = confusion_matrix(data_train.y[data_train.val_idx].detach().cpu().numpy(), y_test_preds[valid_idx].cpu().numpy())

      # impresión matriz de confusión
      plt.subplots(figsize=(6,6))
      sns.set(font_scale=1.4)
      sns.heatmap(conf_mat, annot=True, fmt=".0f", annot_kws={"size": 16}, cbar=False)
      plt.xlabel('Target (true) Class'); plt.ylabel('Output (predicted) class'); plt.title('Confusion Matrix')
      plt.show();

      
      res = classification_report(data_train.y[data_train.val_idx].detach().cpu().numpy(),
                                  y_test_preds[valid_idx].cpu().numpy(),
                                  target_names=['licit', 'illicit'])
      print (res)
      reports[conv_layer] = res

      print(f"Test data fraud cases, percentage: {round (y_test_preds[data_train.test_idx].detach().cpu().numpy().sum() / len(data_train.y[data_train.test_idx]) *100, 2)} % \n")
      fraud_pcnt[conv_layer] = round(y_test_preds[data_train.test_idx].detach().cpu().numpy().sum() / len(data_train.y[data_train.test_idx]) *100, 2)
      
      
    except Exception as e:
      #print(f"Error: {e}. Skipping item: {conv_layer}")
      layers_not_used.append(conv_layer)
      continue
      
print("Finalizado.\n")


#ARMAConv, 0.86
#ClusterGCNConv, 0.93
#FeaStConv, 0.90
#FiLMConv, 0.87
#GATConv, 0.90
#GATv2Conv, 0.91
#GCNConv, 0.91
#GENConv, 0.92
#GeneralConv, 0.90
#GraphConv, 0.90
#HypergraphConv, 0.56
#LEConv, 0.90
#MFConv, 0.90
#ResGatedGraphConv, 0.90
#SAGEConv, 0.90
#SGConv, 0.90
#SuperGATConv, 0.91
#TAGConv, 0.90
#TransformerConv, 0.90
    